#  Deep Learning Tests

## Preparation

In [1]:
# Import necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# Open File
df = pd.read_csv('NYC_Cab_Fare_Wrangled_Time_Distance.csv')

In [3]:
# Show first few rows
df.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,euclidean_distance,...,dayofweek,cold_month,weekend,rush_hour,year,minute,second,15_min_intervals,total_seconds,summer_month
0,0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21,-73.844311,40.721319,-73.841610,40.712278,1,0.009436,...,0,0,0,1,2009,26,21,69,62781,1
1,1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16,-74.016048,40.711303,-73.979268,40.782004,1,0.079696,...,1,1,0,1,2010,52,16,67,60736,0
2,2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00,-73.982738,40.761270,-73.991242,40.750562,2,0.013674,...,3,0,0,0,2011,35,0,2,2100,1
3,3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42,-73.987130,40.733143,-73.991567,40.758092,1,0.025340,...,5,0,1,0,2012,30,42,18,16242,0
4,4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00,-73.968095,40.768008,-73.956655,40.783762,1,0.019470,...,1,1,0,1,2010,51,0,31,28260,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53465062 entries, 0 to 53465061
Data columns (total 22 columns):
Unnamed: 0            int64
key                   object
fare_amount           float64
pickup_datetime       object
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
passenger_count       int64
euclidean_distance    float64
month                 int64
hour                  int64
dayofweek             int64
cold_month            int64
weekend               int64
rush_hour             int64
year                  int64
minute                int64
second                int64
15_min_intervals      int64
total_seconds         int64
summer_month          int64
dtypes: float64(6), int64(14), object(2)
memory usage: 8.8+ GB


In [5]:
# Choose relevant columns
df = df[['key', 'fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'euclidean_distance', 'cold_month', 'weekend', 'rush_hour', '15_min_intervals', 'summer_month']]

### 500,000 rows

In [6]:
df_sub = df.sample(n=500000)
df_sub.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,euclidean_distance,cold_month,weekend,rush_hour,15_min_intervals,summer_month
count,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000,5.000000e+05,500000.000000,500000.000000,500000.000000,500000.000000,500000.000000
mean,11.326031,-73.975557,40.750815,-73.974736,40.751204,1.691840,3.467676e-02,0.419090,0.282736,0.299016,55.591750,0.236896
std,9.550769,0.034416,0.026829,0.038199,0.068023,1.306473,7.377060e-02,0.493411,0.450330,0.457827,26.099228,0.425178
min,2.500000,-74.258628,40.478775,-80.095871,5.650000,1.000000,1.000000e-06,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,-73.992330,40.736561,-73.991607,40.735622,1.000000,1.305657e-02,0.000000,0.000000,0.000000,37.000000,0.000000
50%,8.500000,-73.982116,40.753304,-73.980676,40.753908,1.000000,2.208048e-02,0.000000,0.000000,0.000000,58.000000,0.000000
75%,12.500000,-73.968466,40.767462,-73.965580,40.768387,2.000000,3.899589e-02,1.000000,1.000000,1.000000,78.000000,0.000000
max,459.900000,-73.700982,40.916949,-69.729976,62.143497,6.000000,3.512818e+01,1.000000,1.000000,1.000000,95.000000,1.000000


In [7]:
df_sub.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,euclidean_distance,cold_month,weekend,rush_hour,15_min_intervals,summer_month
2999477,2012-08-17 21:00:27.0000007,14.90,2012-08-17 21:00:27,-73.874523,40.774043,-73.869136,40.720666,1,0.053648,0,0,0,84,1
46094238,2014-01-08 18:07:00.000000185,14.50,2014-01-08 18:07:00,-73.954872,40.765432,-73.989830,40.756545,2,0.036070,1,0,1,72,0
28073526,2013-10-31 08:17:00.000000191,28.33,2013-10-31 08:17:00,-73.912388,40.746285,-73.992225,40.730687,5,0.081346,0,0,1,33,0
41858144,2015-02-21 18:39:29.00000012,4.50,2015-02-21 18:39:29,-73.962593,40.763248,-73.960449,40.759655,1,0.004184,1,1,0,74,0
29066295,2013-12-30 23:57:00.00000028,5.50,2013-12-30 23:57:00,-73.984115,40.761647,-73.974345,40.757470,1,0.010625,1,0,0,95,0


In [9]:
y = np.array(df_sub.fare_amount)
X = np.array(df_sub.iloc[:,3:14])
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Linear Regression                                  

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
fare_predictions = lin_reg.predict(X_train)
lin_mse = mean_squared_error(y_train, fare_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

7.603150293810446

## Decision Tree

In [13]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)
fare_predictions = tree_reg.predict(X_train)
tree_mse = mean_squared_error(y_train, fare_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

3.845549308344409e-16

In [14]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
tree_rmse_scores=np.sqrt(-scores)
print(scores)

[-31.5691684  -31.95237707 -32.04110961 -29.86179506 -27.15530762]


In [15]:
lin_scores = cross_val_score(lin_reg, X_train, y_train, scoring='neg_mean_squared_error', cv=5)
lin_rmse_scores=np.sqrt(-lin_scores)
print(lin_rmse_scores)

[13.54238725 20.04310279  7.66763898  7.2101136   7.23181161]


## Deep Learning Tests

In [25]:
# keras_regression_test requires "from sklearn.model_selection import train_test_split"
def keras_regression_test(X, y, numbers=[128,64], batch_size=32, activation='relu', optimizer='adam', loss='mean_squared_error'):
        
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    # Save the number of columns in predictors: n_cols
    n_cols = X_train.shape[1]

    # Set up the model: model
    model = Sequential()
    
    # Add the first layer
    model.add(Dense(numbers[0], activation=activation, input_shape=(n_cols,)))
    
    # Add addition layers
    for i in range(len(numbers)-1):
        model.add(Dense(numbers[i+1], activation=activation))

    # Add the output layer
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss)

    # Define early_stopping_monitor
    early_stopping_monitor = EarlyStopping(patience=2)

    # Fit the model
    model.fit(X_train, y_train, validation_split=0.3, epochs=30, batch_size=batch_size, callbacks=[early_stopping_monitor])

    # Get score for predictions
    score = model.evaluate(X_test, y_test)
    
    # Get root mean squared error
    rmse = np.sqrt(score)
    
    # Return root mean squared error
    print(rmse)
    
    return model

In [ ]:
# keras_regression_test requires "from sklearn.model_selection import train_test_split"
def keras_regression_final(X, y, numbers=[128,64], batch_size=32, activation='relu', optimizer='adam', loss='mean_squared_error'):
    
    # Save the number of columns in predictors: n_cols
    n_cols = X.shape[1]

    # Set up the model: model
    model = Sequential()
    
    # Add the first layer
    model.add(Dense(numbers[0], activation=activation, input_shape=(n_cols,)))
    
    # Add addition layers
    for i in range(len(numbers)-1):
        model.add(Dense(numbers[i+1], activation=activation))

    # Add the output layer
    model.add(Dense(1))

    # Compile the model
    model.compile(optimizer=optimizer, loss=loss)

    # Define early_stopping_monitor
    early_stopping_monitor = EarlyStopping(patience=2)

    # Fit the model
    model.fit(X, y, epochs=30, batch_size=batch_size, callbacks=[early_stopping_monitor])
    
    return model

In [17]:
keras_regression_test(numbers=[100,50])

Train on 262500 samples, validate on 112500 samples
Epoch 1/30
262500/262500 [==============================] - 8s 30us/step - loss: 64.4959 - val_loss: 21.1828
Epoch 2/30
262500/262500 [==============================] - 8s 31us/step - loss: 26.0319 - val_loss: 21.6086
Epoch 3/30
262500/262500 [==============================] - 8s 31us/step - loss: 23.8584 - val_loss: 18.2701
Epoch 4/30
262500/262500 [==============================] - 8s 31us/step - loss: 23.2856 - val_loss: 26.5943
Epoch 5/30
125000/125000 [==============================] - 1s 9us/step
4.535789480285946


In [22]:
df_sub = df.sample(n=5000000)

In [26]:
y = np.array(df_sub.fare_amount)
X = np.array(df_sub.iloc[:,3:14])
keras_regression_test(X, y, numbers=[100,50])

Train on 2625000 samples, validate on 1125000 samples
Epoch 1/30
2625000/2625000 [==============================] - 81s 31us/step - loss: 24.7949 - val_loss: 24.0757
Epoch 2/30
2625000/2625000 [==============================] - 80s 30us/step - loss: 20.1964 - val_loss: 18.8200
Epoch 3/30
2625000/2625000 [==============================] - 78s 30us/step - loss: 19.7533 - val_loss: 20.6653
Epoch 4/30
1250000/1250000 [==============================] - 14s 11us/step
4.323052388678065


In [27]:
y = np.array(df.fare_amount)
X = np.array(df.iloc[:,3:14])
keras_regression_test(X, y, numbers=[100,50])

Train on 28069157 samples, validate on 12029639 samples
Epoch 1/30
28069157/28069157 [==============================] - 884s 31us/step - loss: 20.2488 - val_loss: 18.5933
Epoch 2/30
28069157/28069157 [==============================] - 966s 34us/step - loss: 19.1320 - val_loss: 19.4809
Epoch 3/30
28069157/28069157 [==============================] - 870s 31us/step - loss: 18.8940 - val_loss: 18.4783
Epoch 4/30
28069157/28069157 [==============================] - 884s 31us/step - loss: 18.8527 - val_loss: 18.3431
Epoch 5/30
28069157/28069157 [==============================] - 868s 31us/step - loss: 18.7319 - val_loss: 22.1989
Epoch 6/30
13366266/13366266 [==============================] - 188s 14us/step
4.345326023849211


In [ ]:
# new instances where we do not know the answer
Xnew, a = make_regression(n_samples=3, n_features=2, noise=0.1, random_state=1)
Xnew = scalarX.transform(Xnew)
# make a prediction
ynew = model.predict(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
	print("X=%s, Predicted=%s" % (Xnew[i], ynew[i]))

In [19]:
model = keras_regression_test(numbers=[100,50], batch_size=36)

Train on 262500 samples, validate on 112500 samples
Epoch 1/30
262500/262500 [==============================] - 7s 28us/step - loss: 65.2405 - val_loss: 19.9077A: 
Epoch 2/30
262500/262500 [==============================] - 7s 27us/step - loss: 24.7549 - val_loss: 18.5223
Epoch 3/30
262500/262500 [==============================] - 8s 30us/step - loss: 24.0240 - val_loss: 19.8624
Epoch 4/30
125000/125000 [==============================] - 1s 10us/step
4.3533494672510376


In [21]:
y = np.array(df.fare_amount)
X = np.array(df.iloc[:,3:14])
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = keras_regression_test(numbers=[100,50])

Train on 262500 samples, validate on 112500 samples
Epoch 1/30
262500/262500 [==============================] - 8s 30us/step - loss: 67.5832 - val_loss: 20.0555
Epoch 2/30
262500/262500 [==============================] - 7s 28us/step - loss: 25.6046 - val_loss: 19.0785
Epoch 3/30
262500/262500 [==============================] - 7s 28us/step - loss: 24.1530 - val_loss: 18.3980
Epoch 4/30
262500/262500 [==============================] - 7s 28us/step - loss: 23.7958 - val_loss: 18.4447
Epoch 5/30
262500/262500 [==============================] - 8s 29us/step - loss: 23.4797 - val_loss: 18.2202
Epoch 6/30
262500/262500 [==============================] - 8s 30us/step - loss: 23.3538 - val_loss: 18.3724
Epoch 7/30
125000/125000 [==============================] - 1s 11us/step
5.668036938678171


In [170]:
batch_sizes = [18, 24, 30, 36]
for batch_size in batch_sizes:
    print('batch size: ', batch_size)
    keras_regression_test(numbers=[100,50], batch_size=batch_size)

batch size:  18
Train on 262500 samples, validate on 112500 samples
Epoch 1/30
262500/262500 [==============================] - 35s 134us/step - loss: 52.1548 - val_loss: 42.1713
Epoch 2/30
262500/262500 [==============================] - 31s 117us/step - loss: 23.1644 - val_loss: 19.1554
Epoch 3/30
262500/262500 [==============================] - 36s 136us/step - loss: 21.5541 - val_loss: 20.0211
Epoch 4/30
125000/125000 [==============================] - 4s 34us/step
4.46488461424254
batch size:  24
Train on 262500 samples, validate on 112500 samples
Epoch 1/30
262500/262500 [==============================] - 26s 99us/step - loss: 52.6370 - val_loss: 28.6305
Epoch 2/30
262500/262500 [==============================] - 29s 112us/step - loss: 21.9789 - val_loss: 24.8884
Epoch 3/30
262500/262500 [==============================] - 26s 100us/step - loss: 21.4638 - val_loss: 19.0206
Epoch 4/30
262500/262500 [==============================] - 24s 93us/step - loss: 21.1327 - val_loss: 18.1729

In [51]:
keras_regression_test(X, y, numbers=[100,50])

Train on 262500 samples, validate on 112500 samples
Epoch 1/30
262500/262500 [==============================] - 10s 38us/step - loss: 55.9240 - val_loss: 23.7459
Epoch 2/30
262500/262500 [==============================] - 10s 38us/step - loss: 22.6585 - val_loss: 22.5477
Epoch 3/30
262500/262500 [==============================] - 11s 43us/step - loss: 22.2504 - val_loss: 21.1804
Epoch 4/30
262500/262500 [==============================] - 10s 38us/step - loss: 21.8715 - val_loss: 21.4971
Epoch 5/30
125000/125000 [==============================] - 2s 12us/step


4.307730960928071

In [54]:
df_sub2 = df.sample(n=5000000)
y2 = np.array(df_sub2.fare_amount)
X2 = np.array(df_sub2.iloc[:,3:12])

In [56]:
y2 = np.array(df_sub2.fare_amount)
X2 = np.array(df_sub2.iloc[:,3:12])

In [59]:
keras_regression_test(X2, y2, numbers=[100,50])

Train on 2625000 samples, validate on 1125000 samples
Epoch 1/30
2625000/2625000 [==============================] - 103s 39us/step - loss: 24.0661 - val_loss: 25.5650
Epoch 2/30
2625000/2625000 [==============================] - 119s 45us/step - loss: 20.2408 - val_loss: 18.2267
Epoch 3/30
2625000/2625000 [==============================] - 116s 44us/step - loss: 20.0077 - val_loss: 21.3367
Epoch 4/30
1250000/1250000 [==============================] - 19s 15us/step


4.300137875702223

Ideas to improve:
1T) Hours into hours minutes seconds (cut month)
2L) Airport 
3L) Boroughs
3T) Weekday, Weekend
4T) Rush Hour 
5T) Seasons
6T) Holidays
7T) Evenings
8T) Weather

### Full Amount (53,000,000 +)

In [164]:
y = np.array(df.fare_amount)
X = np.array(df.iloc[:,3:9])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

### 5,000,000 rows

In [ ]:
keras_regression_test(X2, y2, layers=2, nodes=2, numbers=[128,64])

### More columns added

In [142]:
df_sub2 = df1.sample(n=5000000)
df_sub2.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,euclidean_distance,cold_month,weekend,rush_hour,15_min_intervals,summer_month
29352613,2015-04-07 11:54:02.0000002,14.00,2015-04-07 11:54:02,-73.998657,40.763840,-73.992523,40.735592,1,0.028906,0,0,0,47,0
51380817,2009-04-24 16:46:00.0000001,20.10,2009-04-24 16:46:00,-73.984160,40.727910,-73.986007,40.671605,1,0.056335,0,0,1,67,0
13315395,2012-08-07 21:08:13.0000004,6.10,2012-08-07 21:08:13,-73.991920,40.749328,-74.005050,40.728833,1,0.024340,0,0,0,84,1
21353531,2009-11-28 11:51:00.00000094,49.57,2009-11-28 11:51:00,-73.783647,40.648635,-74.000827,40.747402,1,0.238583,1,1,0,47,0
33664742,2015-04-04 18:58:07.0000002,8.00,2015-04-04 18:58:07,-73.983826,40.738216,-74.002022,40.739578,1,0.018247,0,1,0,75,0


In [150]:
X2 = np.array(df_sub2.iloc[:,3:14])
keras_regression_test(X2, y2, numbers=[100,50])

Train on 10500000 samples, validate on 4500000 samples
Epoch 1/30
10500000/10500000 [==============================] - 730s 70us/step - loss: 21.2324 - val_loss: 19.0953
Epoch 2/30
10500000/10500000 [==============================] - 587s 56us/step - loss: 19.2145 - val_loss: 18.7507
Epoch 3/30
10500000/10500000 [==============================] - 568s 54us/step - loss: 19.0431 - val_loss: 19.0250
Epoch 4/30
10500000/10500000 [==============================] - 553s 53us/step - loss: 18.8915 - val_loss: 18.6707
Epoch 5/30
10500000/10500000 [==============================] - 683s 65us/step - loss: 18.8904 - val_loss: 20.4059
Epoch 6/30
5000000/5000000 [==============================] - 139s 28us/step


4.336311837035749

In [152]:
keras_regression_test(X, y, numbers=[1000])

Train on 262500 samples, validate on 112500 samples
Epoch 1/30
262500/262500 [==============================] - 19s 72us/step - loss: 47.1477 - val_loss: 34.6847
Epoch 2/30
262500/262500 [==============================] - 18s 70us/step - loss: 23.7742 - val_loss: 44.0001
Epoch 3/30
262500/262500 [==============================] - 18s 68us/step - loss: 22.3829 - val_loss: 19.1638
Epoch 4/30
262500/262500 [==============================] - 17s 63us/step - loss: 21.6693 - val_loss: 22.8654
Epoch 5/30
125000/125000 [==============================] - 4s 28us/step


4.3363916224920676

In [10]:
keras_regression_test(numbers=[100,50])

Train on 262500 samples, validate on 112500 samples
Epoch 1/30
262500/262500 [==============================] - 8s 30us/step - loss: 50.6394 - val_loss: 21.0362
Epoch 2/30
262500/262500 [==============================] - 8s 29us/step - loss: 22.6864 - val_loss: 19.8446
Epoch 3/30
262500/262500 [==============================] - 8s 31us/step - loss: 22.0067 - val_loss: 23.2992
Epoch 4/30
125000/125000 [==============================] - 1s 9us/step
4.829188289492332


In [11]:
keras_regression_test()

Train on 262500 samples, validate on 112500 samples
Epoch 1/30
262500/262500 [==============================] - 8s 31us/step - loss: 48.6827 - val_loss: 21.6052
Epoch 2/30
262500/262500 [==============================] - 8s 30us/step - loss: 24.0042 - val_loss: 28.1400
Epoch 3/30
262500/262500 [==============================] - 8s 30us/step - loss: 22.2844 - val_loss: 19.2998
Epoch 4/30
262500/262500 [==============================] - 8s 32us/step - loss: 21.9870 - val_loss: 24.9892
Epoch 5/30
125000/125000 [==============================] - 1s 10us/step
5.119094347493275
